In [1]:
import numpy as np
import pandas as pd
import requests


#I keep this in the comments because I checked to make sure that the page could be requested. It can, but I kept it in case 
#I want to check again later:

#r = requests.get(url)
#r.status_code

In [49]:
#first step is to get the urls. I decided to make a list of each of the teams I would be using. The way they appaear here
#is the same way they appear in the urls 

teams = ["new-york-mets", "philadelphia-phillies", "san-diego-padres", "los-angeles-dodgers", "atlanta-braves", 
         "houston-astros", "new-york-yankees", "chicago-white-sox", "colarado-rockies", "los-angeles-angels", 
         "toronto-blue-jays", "minnesota-twins", "san-fransisco-giants", "texas-rangers", "boston-red-socks",
         "chicago-cubs", "seattle-mariners", "detriot-tigers", "st-louis-cardinals", "washington-nationals",
         "arizona-diamondbacks", "cincinnati-reds", "miami-marlins", "milwaukee-brewers", "tampa-bay-rays",
         "kansas-city-royals", "pittsburgh-pirates", "cleveland-guardians", "baltimore-orioles", "oakland-athletics"]

link = []

In [50]:
#Now to actually get the urls. This is a simple for loop to get each team into a link and then add it to a list. I used to 
#set(link) to make sure every entry was unique because I kept on getting https://www.spotrac.com/mlb/i/payroll/2022/ 30 
#times? I'm not really sure why and I know there's probably a better fix, but this seemed easier

for i in teams:
    link.append("https://www.spotrac.com/mlb/" + i + "/payroll/2022/")

link = set(link)

In [ ]:
#checking to make sure link actually produces the output I want, once again because I had trouble

link

In [52]:
#This gives me the tables I want. It reads all the tables in a list, with each webpage being the bigger part of the list. 
#It essentially makes a list within a list

tablelist = []
for l in link:
    tablelist.append(pd.read_html(l))

In [53]:
#Now I'm concatinating the dataframes together. I only want the first table in each list, so I'm concatinating those in this
#for loop

mlbPayroll2022v1 = pd.DataFrame()
for t in tablelist:
    mlbPayroll2022v1 = pd.concat([mlbPayroll2022v1, t[0]])

In [55]:
mlbPayroll2022v1

,Active Players (28),Age,Pos.,Status,Base Salary,Signing Bonus,Incentives,Payroll Salary,Adj. Salary,Payroll %,Lux. Tax Salary,Active Players (29),Active Players (842),Active Players (27),Active Players (25),Active Players (31),Active Players (30)
0,Chad Pinder,30,LF,Arb 3,"$2,725,000",-,-,"$2,725,000","$2,725,000",4.47,2725000,NaN,NaN,NaN,NaN,NaN,NaN
1,Tony Kemp,30,2B,Arb 2,"$2,250,000",-,-,"$2,250,000","$2,250,000",3.69,2250000,NaN,NaN,NaN,NaN,NaN,NaN
2,Stephen Vogt,37,C,Vet,"$850,000",-,"$100,000","$950,000","$950,000",1.56,850000,NaN,NaN,NaN,NaN,NaN,NaN
3,Sean Murphy,27,C,Pre-Arb,"$725,000",-,-,"$725,000","$725,000",1.19,725000,NaN,NaN,NaN,NaN,NaN,NaN
4,Seth Brown,29,LF,Pre-Arb,"$705,000",-,-,"$705,000","$705,000",1.16,705000,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,Mark Mathias,27,2B,Pre-Arb,"$700,000",-,-,"$700,000","$192,300",0.12,192300,NaN,NaN,NaN,NaN,NaN,NaN
24,Jesus Tinoco,27,RP,Pre-Arb,"$700,000",-,-,"$700,000","$165,378",0.10,165378,NaN,NaN,NaN,NaN,NaN,NaN
25,Josh Jung,24,3B,Pre-Arb,"$700,000",-,-,"$700,000","$99,996",0.06,99996,NaN,NaN,NaN,NaN,NaN,NaN
26,A.J. Alexy,24,RP,Pre-Arb,"$700,000",-,-,"$700,000","$73,074",0.04,73074,NaN,NaN,NaN,NaN,NaN,NaN


In [54]:
#for the sake of my sanity and time I made a csv of the top output so it wouldn't take 20 minutes to go through again

mlbPayroll2022v1.to_csv ("mlbPayroll2022v1.csv")

In [56]:
#Now we start the data cleaning process

mlbPayroll = pd.read_csv("mlbPayroll2022v1.csv")

In [57]:
#dropping all the variables I don't want

mlbPayroll = mlbPayroll.drop(["Base Salary", "Signing Bonus", "Incentives", "Payroll Salary", "Payroll %", 
                              "Lux. Tax Salary", "Unnamed: 0"], axis = 1)

In [58]:
#The table's in a wide format with a lot of missing values, and the player's names aren't in the same column, so I wanted
#to make it long with the player's names in the same column

mlbPayroll = pd.melt(mlbPayroll, id_vars = mlbPayroll.columns[1:5], value_name = "Active Players")

In [59]:
#The long table has multiple extra rows with extranous information. I know the table only has 4100 values, not 41000, so 
#I've removes the null values since those are repeat values with no new information

mlbPayroll2022 = mlbPayroll.dropna()

In [60]:
#Once again dropping the variable I don't want and switching the order of the columns because I want the names to go first

mlbPayroll2022 = mlbPayroll2022.drop("variable", axis = 1)
mlbPayroll2022 = mlbPayroll2022.iloc[:, [4, 0, 1, 2, 3, ]]

In [61]:
#checking the final output to make sure it looks how I want it 
mlbPayroll2022

,Active Players,Age,Pos.,Status,Adj. Salary
0,Chad Pinder,30,LF,Arb 3,"$2,725,000"
1,Tony Kemp,30,2B,Arb 2,"$2,250,000"
2,Stephen Vogt,37,C,Vet,"$950,000"
3,Sean Murphy,27,C,Pre-Arb,"$725,000"
4,Seth Brown,29,LF,Pre-Arb,"$705,000"
...,...,...,...,...,...
26822,Albert Abreu,26,RP,Pre-Arb,"$707,500"
26823,Clarke Schmidt,26,RP,Pre-Arb,"$481,545"
26824,Scott Effross,28,RP,Pre-Arb,"$250,582"
26825,Oswaldo Cabrera,23,2B,Pre-Arb,"$188,454"


In [62]:
mlbPayroll2022.to_csv ("mlbPayroll2022.csv")